# Introduction to Photonic Band Gap fiber class.

The class PBG is used to create geometries describing Photonic Band Gap fibers and assist in calculating their modes.  In this tutorial we describe how to create PBG class objects for these geometries and use these objects to find the desired modes.

## Setup: Parameters

Photonic Band Gap fibers feature a hexagonal lattice of tubes of high refractive index embedded in a cladding of lower refractive index.  Below we see two examples of such fibers.  The image is taken from [1].

In [ ]:
from IPython.display import Image
Image(filename='basic_fibers.png') 

The parameters necessary for recreating these fibers are stored in the directory "fiber_dicts" inside the main "pbg" directory.  The parameters for fiber (a) are shown below:

In [ ]:
from fiberamp.fiber.microstruct.pbg.fiber_dicts.lyr7cr1 import params

params

The meaning of these parameters is as follows:

Geometric Paramters:
- p: refers to the number of sides of the polygonal lattice.  For a hexagonal lattice we use p=6.
- layers: refers to the number of layers of tubes.  For (a) there are a total of 7 layers.
- skip: indicates number of layers of tubes to be skipped to make the core region.  For (a) we skip 1 layer, while for (b) we skip 2.
- sep: Identical to $\Lambda$ above; indicates the distance separating layers of tubes
- r_tube: the radius of the tubes, equal to d/2 where d appears above.
- r_core: radius of the core region.  This is the area inside the lattice formed by skipping layers.
- r_fiber: The radius of the fiber as a whole.  More generally this can be any radius after which refractive profile is homogeneous.
- scale: factor by which to scale the fiber parameters to make a non-dimensional geometry.  Frequently chosen to make the core region unit radius.
- n_tube, n_clad: the refractive indices of the tube and cladding material respectively.
- n_core, n_buffer, n_outer: the refractive index present in the respective regions.  We will often have n_core the same as n_clad as it is frequently a subregion of the cladding.  The index n_outer must be the same as n_buffer as these appear in the homogeous region.
- n0 : Base refractive index used in the refractive index function V.  Must be same as n_outer and n_buffer.

Mesh Parameters:
- t_buffer, t_outer: desired thickness of the buffer and PML regions respectively.
- alpha: PML parameter.
- pml_, air_, tube_, clad_, core_ maxh: maximum element diameter for mesh on respective regions.

If you wish to create a different fiber, copy the file and alter parameters to your liking and rename it.  Note that the name 'lyr7cr1' indicates the fiber has seven layers of tubes and a core of one missing layer of tubes.  The file containing the parameters for (b) is called 'lyr6cr2' conforming to this.

## Creating a PBG fiber instance

We can instantiate a PBG class object by loading the class and a parameter dictionary like the one described above, and passing the dictionary to the class:

In [ ]:
from fiberamp.fiber.microstruct.pbg import PBG
from fiberamp.fiber.microstruct.pbg.fiber_dicts.lyr7cr1 import params

In [ ]:
A = PBG(params)

In [ ]:
A.__dict__

Now we can visualize the mesh as follows:


In [ ]:
# import netgen.gui
from ngsolve.webgui import Draw
import ngsolve as ng

In [ ]:
Draw(A.mesh)

The fiber shown in (b) is created similarly:

In [ ]:
from fiberamp.fiber.microstruct.pbg.fiber_dicts.lyr6cr2 import params

In [ ]:
B = PBG(params)

In [ ]:
Draw(B.mesh)

## Mode finding

Since the PBG class is derived from ModeSolver, we can use the facilites there to look for modes of the fiber.  Doing so would look like the following:

In [ ]:
z2, y2, yl2, beta2, P2, _ = B.leakymode(2, ctr= 1.242933 - 2.471929e-09j, rad=.01, npts=4, nspan=2)


In [ ]:
y2.gridfun().vec.FV().NumPy()

We can visualize the results:

In [ ]:
Draw(y2.gridfun())

## Other geometries

The PBG class can easily construct more complex geometries such as the one shown below:

In [ ]:
from IPython.display import Image
Image(filename='pattern_fiber.png') 

This fiber is encoded in a similar way to (b), but the pattern seen is created by passing a list that determines which tubes are removed.  Here we describe how to create that list.  

First, we setup some rules for navigation.  We assume the origin is at the center of the fiber and orthogonal x and y axes are centered at that origin with x pointing due east, (hence falling along one vertex of the underlying hexagon).  We then say that the layers are indexed starting at 0 with a single tube at the origin, followed by a layer (at distance $\Lambda$) at index 1 consisting of a hexagon of six tubes, followed by a layer at index 2 of 12 tubes, etc.  We order the tubes in a given layer counter-clockwise starting with the tube at the vertex intersecting the x-axis.  

Note that in the above geometry we have the same number of layers and core size as in geometery (b) namely 6 layers of tubes and 2 layers skipped to make the core.  The first two layers of tubes are complete, but the remaining layers have some missing.  To encode the first two layers, we set-up a list like this:


In [ ]:
pattern = [[1,1,1,1,1,1,1,1,1,1,1,1],
           [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]]

The list has as first entry a list consisting of 12 ones, one for each of the tubes in the first (non-core) layer of tubes that is present.  The second entry is a list of 18 ones, a one for each tube that appears in the second (non-core) layer of tubes.  So far, only 1's have appeared since no tube has been missing.

For the next layer we begin at the tube intersecting the x axis and go counter-clockwise.  The first tube appears, so that gives a 1, as does the second, another 1, while the third is missing, so we put a 0.  It then continues with three 1's, then a 0, then three more 1's, etc.  Thus we append the following:

In [ ]:
l3 = [1,1,0,1,
      1,1,0,1,
      1,1,0,1,
      1,1,0,1,
      1,1,0,1,
      1,1,0,1]
pattern.append(l3)
pattern

We continue in this way for the remaining three layers:

In [ ]:
l4 = [1,0,1,1,0,
      1,0,1,1,0,
      1,0,1,1,0,
      1,0,1,1,0,
      1,0,1,1,0,
      1,0,1,1,0]

l5 = [0,1,1,0,1,1,
      0,1,1,0,1,1,
      0,1,1,0,1,1,
      0,1,1,0,1,1,
      0,1,1,0,1,1,
      0,1,1,0,1,1]

l6 = [1,1,0,1,1,0,1,
      1,1,0,1,1,0,1,
      1,1,0,1,1,0,1,
      1,1,0,1,1,0,1,
      1,1,0,1,1,0,1,
      1,1,0,1,1,0,1]
pattern.append(l4)
pattern.append(l5)
pattern.append(l6)

Since everything about this fiber is the same as (b) in the first case, we can use the parameters from (b) but change the "pattern" attribute from the empty list to our new list:

In [ ]:
params['pattern'] = pattern

In [ ]:
C = PBG(params)

In [ ]:
Draw(C.mesh)

# References

1) Murao, Tadashi & Saitoh, Kunimasa & Koshiba, Masanori. (2011). Multiple resonant coupling mechanism for suppression of higher-order modes in all-solid photonic bandgap fibers with heterostructured cladding. Optics express. 19. 1713-27. 10.1364/OE.19.001713. 